In [50]:
%run utils.ipynb
from bs4 import BeautifulSoup
import requests
import sys
import time
import pandas as pd
import urllib.robotparser as urobot
from tqdm import tqdm_notebook as tqdm

header = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_5) AppleWebKit/605.1.15 (KHTML, like Gecko) Safari/605.1.15'}
urls_test = ["https://www.submarino.com.br"]

urls = ["https://www.americanas.com.br",
        "https://www.shoptime.com.br", 
        "https://www.submarino.com.br",
        "https://www.magazineluiza.com.br",
        "https://www.casasbahia.com.br",
        "https://www.extra.com.br",
        "https://www.pontofrio.com.br",
        "https://www.saraiva.com.br",
        "https://www.amazon.com.br",
        "https://www.ricardoeletro.com.br",
        "https://www.kabum.com.br",
        "https://br.gearbest.com",
        "https://www.ibyte.com.br",
        "https://www.fastshop.com.br"]


In [53]:
#Class definition of the bfs_crawler
class bfs_crawler:
    def __init__(self, url):
        self.url = url                
        self.robotParser = getRobot(url)      #robot parser to check if a link is valid or not
        self.links_list = []                  #list of allowed links
        self.file_name  = set_file_name(url)  #name of the files generate by the crawler 
        
#     def __init__(self):
#         self.url = None
#         self.robotParser = None
#         self.links_list = []
    
    def get_links(self):
        actual_link = self.url
        link_count = 0
        index = 0
        
        while (len(self.links_list) < 1000):
            try:
                if("http" not in actual_link):
                    actual_link = "http://" + actual_link
                req = requests.get(actual_link, headers=header)
                
                if(req.status_code == 200):    
                    soup = BeautifulSoup(req.text, "html.parser")
                    pageLinks = soup.findAll("a")
                    for a in pageLinks:
                        link = a["href"]
                        print(link)
                        if(self.robotParser.can_fetch("*", link) and "http" in link):
                            #print("{} - {}".format(link_count, link))
                            self.links_list.append(link)
                            link_count += 1
                        
                    
            except Exception:
                print("Link inválido!")
            finally: 
                if(index >= len(self.links_list)):
                    print("END")
                    return
                actual_link = self.links_list[index]
                index += 1
                sleep(1)
                
    #saves the links as csv 
    def save_as_csv(self, file_name):
        df = pd.DataFrame(self.links_list, columns=["links"])
        df.to_csv((file_name +'.csv'),header=True, index=False, encoding='utf-8')
    
    #save the robots from a url
    def save_robot(self):
        robot_url = self.url + "/robots.txt"
        req = requests.get(robot_url,headers=header)
        content = req.text
        save_file(content, "robots", self.file_name)
    
    #getters and setters methods
    def get_url(self):
        return self.url
    def set_url(self, url):
        self.url = url
    def get_links_list(self):
        return self.links_list
    def set_links_list(self, links):
        self.links_list = links

In [55]:
for url in urls: 
    crawler = bfs_crawler(url)
    crawler.save_robot()
    #print(crawler.robotParser.text)
    #crawler.get_links()
    #crawler.save_as_csv(file)